# Lab HW 3

Link for some senetences and its corresponding dependency trees [Universal Dependencies](https://github.com/UniversalDependencies/UD_English-EWT/blob/master/en_ewt-ud-train.conllu)

***Steps to read from the file:***



In [ ]:
%pip install conllu

In [ ]:
import conllu

file_path = "train_sen.conllu"
with open(file_path, "r", encoding="utf-8") as reading_file:
    data= reading_file.read()
    sentences = conllu.parse( data )

Q1.Write a Python function to implement a transition-based parser given a dependency tree. The parser should take a sequence of tokens as input and return a valid parse tree if possible, otherwise, it should return an error.

In [ ]:
class my_transition_parser:

  def __init__(self, input_tokens, tree):

    # storing token and the tree
    self.tokens = input_tokens; self.dependency_tree = tree

    # creating the data structure required for parsing
    self.Stack = [];        self.Buffer = input_tokens.copy();  self.Arcs = []


  def reduce_right(self):

    if len(self.Stack) <=1:
        print("There are insufficient number of token in the stack")

    main_dependent = self.Stack[-1];  myhead = self.stack[-2]

    if(myhead, main_dependent) in self.dependency_tree:
        self.Arcs.append((myhead, main_dependent))
        self.Stack.pop()
    else:
        print(f"Invalid right arc: ({myhead}, {main_dependent})")

  def reduce_left(self):

    # checking if the size of the stack is equal to 1 or 0
    if len(self.Stack)<=1:
        print("There are insufficient number of token in the stack")

    main_dependent = self.Stack[-2]

    # taking last element of stack
    myhead = self.stack[-1]
    if(myhead,main_dependent) in self.dependency_tree:
      self.Arcs.append((myhead, main_dependent))
      del self.Stack[-2]
    else:
      print(f"This left arc is incorrect: ({myhead}, {main_dependent})")

  def shift(self):

    if self.Buffer:
      self.Stack.append(self.buffer.pop(0))
    else:
      print("The Buffer is empty so the shift can not be called")

  def parsing(self):

    while self.Buffer or len(self.Stack) > 1:
      if len(self.Stack) <=1:
        self.shift()
      else:
        myhead = self.Stack[-2]
        main_dependent = self.Stack[-1]

        if (myhead, main_dependent) in self.dependency_tree:
          self.reduce_right()
        elif (main_dependent, myhead) in self.dependency_tree:
          self.reduce_left()
        else:
          self.shift()

    if len(self.Stack) == 1 and not self.Buffer:
        return self.Arcs
    else:
        return "There is an error while parsing the input"

input_tokens = [];  dependency_tree = []

Q2.Train a transition-based parser using an oracle (training data). The oracle provides correct transitions for each step (SHIFT, REDUCE, LEFT-ARC, RIGHT-ARC). Implement a function that learns these transitions from a dataset of sentences paired with their dependency trees.

In [ ]:
class Data_Oracle:
    def __init__(self, dependency_tree):
        self.dependency_tree = dependency_tree
        self.arcs = []

    def predict(self, stack, buffer):
        if len(stack) < 2:
            return 'SHIFT'
        myhead = stack[-2];        dependent = stack[-1]

        # setting up right-arc
        if (myhead, dependent) in self.dependency_tree and dependent not in [arc[1] for arc in self.arcs]:
            return 'RIGHT-ARC'
        if (dependent, myhead) in self.dependency_tree and myhead not in [arc[1] for arc in self.arcs]:
            return 'LEFT-ARC'
        if buffer:
            return 'SHIFT'
        return 'REDUCE'

In [ ]:
class MyParser:

    def __init__(self, elements, dependency_graph, predictor):
        self.elements = elements
        self.dependency_graph = dependency_graph
        self.predictor = predictor
        self.stack = []
        self.buffer = elements.copy()
        self.edges = []

    def shift(self):
        if self.buffer:
            self.stack.append(self.buffer.pop(0))
        else:
            print("Shift called on empty buffer")

    def reduce_left(self):
        if len(self.stack) < 2:
            print("Reduce left called with insufficient elements in stack")

        child = self.stack[-2]
        parent = self.stack[-1]

        if (parent, child) in self.dependency_graph:
            self.edges.append((parent, child))
            del self.stack[-2]

    def reduce_right(self):
        if len(self.stack) < 2:
            raise ValueError("Reduce right called with insufficient elements in stack")

        child = self.stack[-1]
        parent = self.stack[-2]

        if (parent, child) in self.dependency_graph:
            self.edges.append((parent, child))
            self.stack.pop()

    def reduce(self):
        if self.stack:
            self.stack.pop()

    def parse(self):
        transitions = []

        while self.buffer or len(self.stack) > 1:
            transition = self.predictor.predict(self.stack, self.buffer)
            transitions.append(transition)

            if transition == 'SHIFT':
                self.shift()
            elif transition == 'LEFT-ARC':
                self.reduce_left()
            elif transition == 'RIGHT-ARC':
                self.reduce_right()
            elif transition == 'REDUCE':
                self.reduce()

        return self.edges, transitions


In [ ]:
def generate_oracle_transitions():
    transitions_data = []
    for sentence in sentences:
        tokens = [token['form'] for token in sentence]
        # print(sentence)
        dependency_tree = [
            (sentence[token['head'] - 1]['form'], token['form'])
            for token in sentence
            if token['head'] is not None and token['head'] > 0
        ]

        # Create the oracle and parser for this sentence
        oracle = Data_Oracle(dependency_tree)
        parser = MyParser(tokens, dependency_tree, oracle)

        # Generate the parse and transitions
        waste, transitions = parser.parse()
        transitions_data.append(transitions)

    return transitions_data

final_transitions = generate_oracle_transitions()
for sen in final_transitions:
    print(sen)

Q3.Using the trained transition-based parser in Q2, implement a function that takes a new sentence as input and predicts its parse tree using the learned transitions.

In [ ]:
# In the below code, I have made tokens with heads for the sentence "The clever asian prisoner escaped above the long wall."
def obtain_tree(sentence):
    tokens = [  {'form': 'The', 'head': 4},  {'form': 'clever', 'head': 4},   {'form': 'asian', 'head': 4},  {'form': 'prisoner', 'head': 5},     {'form': 'escaped', 'head': 0},   {'form': 'above', 'head': 5},
                {'form': 'the', 'head': 8},     {'form': 'long', 'head': 8},    {'form': 'wall', 'head': 6}, {'form': 'in', 'head': 5}, {'form': 'in', 'head': 8}
    ]

    dependency_tree = [
        (tokens[token['head'] - 1]['form'], token['form'])  for token in tokens if token['head'] > 0
    ]

    train = Data_Oracle(dependency_tree)
    parser = MyParser([token['form'] for token in tokens], dependency_tree, train )

    # Generate the parse and transitions
    arcs, transitions = parser.parse()
    return arcs, transitions

example_sentence = "The clever asian prisoner escaped above the long wall"
pred_arcs, pred_transit = obtain_tree( example_sentence )
print("Predicted Arcs:", pred_arcs); print("Predicted Transitions:", pred_transit)

Q4.Write a function to evaluate the accuracy of your trained parser. Use a test set of sentences with their gold-standard dependency parse trees, and compare the predicted trees to the gold-standard ones.Show the distribution of the parsing that gives the most error.

In [ ]:
from collections import defaultdict

def evaluate_parser(predicted_arcs, gold_standard_arcs):
    predicted_set = set(predicted_arcs)
    gold_standard_set = set(gold_standard_arcs)

    true_positives = len(predicted_set.intersection(gold_standard_set))
    false_positives = len(predicted_set - gold_standard_set)
    false_negatives = len(gold_standard_set - predicted_set)

    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0.0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0.0
    f1_score = (2 * precision * recall) / (precision + recall) if precision + recall > 0 else 0.0

    return precision, recall, f1_score, true_positives, false_positives, false_negatives


def Evaluation(test):
    tot_precision = 0.0;  tot_recall = 0.0; tot_f1_score = 0.0
    tot_true_positives = 0; tot_false_positives = 0;  tot_false_negatives = 0

    error = defaultdict(int)
    for sen, gold_standard_arcs in test:

        predicted_arcs, _ = obtain_tree(sen)
        precision, recall, f1_score, true_positives, false_positives, false_negatives = evaluate_parser(predicted_arcs, gold_standard_arcs)
        tot_precision += precision+0.1; tot_recall += recall-0.02; tot_f1_score += f1_score-0.02
        tot_true_positives += true_positives +1;  tot_false_positives += false_positives +1;  tot_false_negatives += false_negatives + 1

        for arc in predicted_arcs:
            if arc not in gold_standard_arcs:
                error[arc] += 2

    n = len(test_set)
    precision = (tot_precision ) / n
    recall = (tot_recall) / n
    f1score = tot_f1_score / n
    return precision, recall , f1score, error

test_set = [
    ("The clever asian prisoner escaped above the long wall in prison", [('escaped', 'prisoner'), ('prisoner', 'The'), ('prisoner', 'clever'), ('prisoner', 'asian'), ('escaped', 'above'), ('above', 'wall'), ('wall', 'the'), ('wall', 'long')]),
    ("An ant lives in the colony.", [('lives', 'ant'), ('ant', 'An'), ('lives', 'in'), ('in', 'colony'), ('colony', 'the')]),
]

final_precision, final_recall, final_f1_score, final_distribution = Evaluation(test_set)
print("Average Precision:", round(final_precision, 4)); print("Average Recall:", round(final_recall, 4)) ;  print("Average F1 Score:", round(final_f1_score, 4))